In [17]:
from com.ibm.watson import get_mongo_client, DOCS_DIR, DESKTOP
from com.ibm.watson.utils import excel_utils
import os, re, sys, pymongo, openpyxl

In [3]:
client = get_mongo_client(port=27017)
db = client["WATSON"]

In [4]:
msk_trials = [ x.strip() for x in open(os.path.join(DESKTOP, "trial_stats", "trials_diff_from_MSK.txt")) ]
print len(msk_trials)

104


In [8]:
existing_trials = db["CTGOV"].find({"NCT_ID" : {"$in" : msk_trials}})
print existing_trials.count()
existing_trials_ids = [ x["NCT_ID"] for x in existing_trials]

31


In [16]:
not_existing_trials = set([ nct_id for nct_id in msk_trials if nct_id not in existing_trials_ids ])
print len(set(msk_trials))
print len(existing_trials_ids)
print len(not_existing_trials)
print "\n".join(not_existing_trials)

41
31
10
NCT01813214
NCT02427893
NCT02465060
NCT02263898
NCT02721459
NCT02475213
NCT02608034
NCT01871311
NCT01651013
NCT02915666


In [19]:
workbook = openpyxl.load_workbook(os.path.join(DESKTOP, "trial_stats", "Trials_diff_from_MSK.xlsx"))
worksheet = workbook.get_active_sheet()
for i, row in enumerate(worksheet.rows):
    if i > 0:
        nct_id = row[1].value.strip()
        study = db["CTGOV"].find_one({"NCT_ID" : nct_id})
        if study:
            row[2].value = "Y"
            row[3].value = "Y" if study["metadata"]["STATUS"] == "ACCEPTED" else "N"
        else:
            row[2].value = "N"
            row[3].value = "N"
            
workbook.save(os.path.join(DESKTOP, "trial_stats", "Trials_diff_from_MSK_EDITED.xlsx"))

/Users/singhv/Library/Python/2.7/lib/python/site-packages/openpyxl/workbook/workbook.py:102: UserWarning: Call to deprecated function or class get_active_sheet (Use the .active property).
  def get_active_sheet(self):
